In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# Path to folder with test data
There must be to directories inside: "clean" and "noisy"

In [2]:
test_path = 'data/val/val/'

#you can fill the list below with indeces of data elements to visualize
list2visualize = [] #[0,1,2]

### Cuda device checking

In [3]:
device = ''
if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"
print(f'Your device is "{device}"')

Your device is "cuda:0"


### Form a list of directories which contain train or val samples

In [4]:
def get_clean_data_paths(path: str):
    """
    input: path to train or val dir with 'clean' and 'noise' dirs
    output: list of paths to clean data
    """
    clean = 'clean/'
    list_clean = []
    with os.scandir(path+clean) as entries:
        for entry in entries:
            if entry.is_dir():
                list_clean.append(entry.path)
    return sorted(list_clean)

def check_data_paths(paths_list, show_cnt = 10):
    """
    outputs number of input list elements and their values
    """
    print(f'input list length = {len(paths_list)}')
    for i, elem in enumerate(paths_list):
        print(elem)
        if i==show_cnt:
            break
    pass

### Load, transpose and check data

In [5]:
def get_data_from_clean_data_paths(clean_data_paths: list):
    """
    Loading numpy arrays situated on input list paths.
    input: paths to clean data
    output: loaded clean and noisy data to lists of np.ndarrays    
    """
    clean_data_list = []
    noisy_data_list = []

    #scans every dir path
    for path in clean_data_paths:
        #scans every file in current dir
        with os.scandir(path) as entries:
            for entry in entries:
                if entry.is_file():
                    clean_path = entry.path
                    noisy_path = entry.path.replace('clean', 'noisy',1)

                    clean_data_list.append(np.load(clean_path).T)
                    noisy_data_list.append(np.load(noisy_path).T)
    return clean_data_list, noisy_data_list

def check_loaded_data(data1_list, data2_list, check_part=0.1, check_shapes=True, print_shapes=False, check_data=False):
    """
    Help checking loaded data: if data from 2 lists has same shape and doesn't contain nans, zeros only and so on
    """
    
    cnt2check = int(max(1, len(data1_list)*check_part))
    samples2check = np.random.choice(range(len(data1_list)), cnt2check, replace=False)
    
    different_shape = False
    for index in samples2check:
        if data1_list[index].shape != data2_list[index].shape:
            different_shape = True
            print(f'Difference in shape is detected!'
                 f'\t{data1_list[index].shape} vs {data2_list[index].shape}')
            break
        if print_shapes:
            print(data1_list[index].shape)
    
    if check_data:
        for index in samples2check:
            print(f'mean1 = {np.mean(data1_list[index]):5.3}, std1 = {np.std(data1_list[index]):5.3}\t'
                  f'mean2 = {np.mean(data2_list[index]):5.3}, std2 = {np.std(data2_list[index]):5.3}')
        
def get_data_statistics(data):
    """
    Prints some data statistics and plots histogram of data lengths
    """
    dataLens = []
    for d in data:
        dataLens.append(d.shape[1])
    dataLens = np.array(dataLens)
    maxLen = np.max(dataLens)
    minLen = np.min(dataLens)
    meanLen = np.mean(dataLens)
    stdLen = np.std(dataLens)
    print(f'Data length params: min={minLen}, max={maxLen}, mean={meanLen}, std={stdLen}')
    plt.figure(figsize=(10,10))
    sns.histplot(data=dataLens)

### We will use PyTorch functionality for batch construction:
  - custom Dataset class to store data with different sizes,
  - customized collate_fn to form batches of different shape elements in DataLoader.

In [6]:
class DetectionDS(Dataset):
    """
    Dataset class.
    Maintains
        data - clean and noisy data
        targets - noise residuals of the same size as noisy or clean data
    """
    def __init__(self, clean, noisy):
        #initialize class object
        
        #concatenate clean and noisy data lists
        self.data = noisy
        #generate corresponding targets
        self.targets = [noisy[i]-clean[i] for i in range(len(clean))]
        
    def __len__(self):
        #standard interface function for Dataset
        if self.data != None:
            return len(self.data)
        
    def __getitem__(self, idx):
        #standard interface function for Dataset
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = [self.data[idx], self.targets[idx]] #{'mel_data': self.data[idx], 'target': self.targets[idx]}
        return sample

In [7]:
def collate_function(batch):
    """
    A customization of default PyTorch collate_fn function
    Forms batch from items of different size:
        looks for min item length and then randomly 
        cuts parts of min length from each item
    """
    #extract data from input batch
    data = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    min_len = 100000
    for elem in data:
        min_len = min(min_len, elem.shape[1])
    
    #slicing data and targets to size of minimum element
    temp_pairs = [random_cut(elem[0], elem[1], min_len) for elem in zip(data, targets)]
    data = [elem[0] for elem in temp_pairs]
    targets = [elem[1] for elem in temp_pairs]
    return [torch.tensor(data).float(), torch.tensor(targets).float()]
        
    
def random_cut(array1, array2, slice_len):
    """
    Help function to randomly cut simultaneously 2 arrays through slicing
    """
    start_cut_range = array1.shape[1] - slice_len
    random_start = np.random.randint(0, start_cut_range+1)

    return (array1[:, random_start: random_start+slice_len], array2[:, random_start: random_start+slice_len])

In [8]:
def get_DataLoader(clean_data_list, noisy_data_list, batch_size, shuffle=True):
    """
    Returns dataloaders for further training and evaluating
    Input:
        clean_data_list - list of numpy arrays with clean data
        noisy_data_list - list of numpy arrays with noisy data
        batch_size - defines batch size
    """
    dataset = DetectionDS(clean_data_list, noisy_data_list)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, 
                            collate_fn=collate_function, pin_memory=True)
    return dataloader

### Model definition
For noise reduction problem we will use 15-layer DnCNN-like CNN with BatchNorm layers.

In [9]:
class NoiseReductionModel(nn.Module):
    """
    Model definition.
    It is fully-convolutional network, where objective is noise residual part.
    If Y is residual, X is clean data and Y' is noisy data, then CNN output will be 
        Y = Y'- X,
    and accordingly our objective will be computed as 
        X = Y' - Y
    Layers summary:
        There are 20 layers:
            1 Conv1D layer->ReLU,
            18 Conv1D layers->BatchNorm->RelU,
            1 Conv1D layer
    """
    def __init__(self, nLayers=20):
        super(NoiseReductionModel, self).__init__()
        #DnCNN like architecture
        self.repeated_layers = nLayers-2
        self.conv_list = nn.ModuleList()
        for nLayer in range(self.repeated_layers):
            self.conv_list.append(nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1))
            self.conv_list.append(nn.BatchNorm1d(64))
            self.conv_list.append(nn.ReLU())
            if nLayer>0 and nLayer%4==0:
                self.conv_list.append(nn.Dropout(0.5))
        
        self.ConvFirst = nn.Conv1d(in_channels=80, out_channels=64, kernel_size=3, padding=1)
        self.ConvLast = nn.Conv1d(in_channels=64, out_channels=80, kernel_size=3, padding=1)
        
        
    def forward(self, inputs):
        x = F.relu(self.ConvFirst(inputs))
        
        for layer in self.conv_list:
            x = layer(x)
        
        x = self.ConvLast(x)
        
        return x

### Auxiliary functions for training epoch and evaluating:

In [10]:
def Evaluate(model, dataloader, device):
    """
    Computes output for data in dataloader and returns loss.
    Input: 
        model - trained model, 
        dataloader - DataLoader with evaluating data,
        device - chosen device
    """
    model.eval()
    criterion = nn.MSELoss(reduction='mean')
    running_loss = 0.
    handled_samples = 0
    for batch_number, data in enumerate(dataloader):
        samples, targets = data[0], data[1]
        samples, targets = samples.to(device) , targets.to(device)

        predictions = model(samples)
        targets = targets
        samples = samples
        mel_predictions = samples - predictions
        mel_targets = samples - targets
        
        running_loss += criterion(mel_predictions, mel_targets).item()
        handled_samples += targets.shape[0]
    
    return running_loss/handled_samples

# Denoising evaluation
Loading best trained model and compute its MSE loss

In [11]:
loaded_denoising_model = NoiseReductionModel(15)
model_state_dict = torch.load(f'denoiser_epoch4.pth')
loaded_denoising_model.load_state_dict(model_state_dict)
loaded_denoising_model.to(device);

In [12]:
test_clean_paths_list = get_clean_data_paths(test_path)
test_clean_data, test_noisy_data = get_data_from_clean_data_paths(test_clean_paths_list[:])
test_dataloader = get_DataLoader(test_clean_data, test_noisy_data, batch_size=1, shuffle=False)

In [13]:
test_loss = Evaluate(loaded_denoising_model, test_dataloader, device)
print(f'MSE = {test_loss:6.4}')

MSE = 0.05488


# Results visualisation
For each element with the index in list2visualize calculate MSE loss and visualize noisy->predicted->clean data

In [14]:
def EvaluateAndVisualize(model, clean_data, noisy_data, device, list_of_indeces_to_check):
    """
    Computes output for data of certain indeces, returns loss and draw heatmaps of data.
    Input: 
        model - trained model,
        clean_data - clean data,
        noisy_data - noisy data,
        device - chosen device,
        list_of_indeces_to_check - list of indeces of data to compute loss and draw figures
    """
    model.eval()
    dataset = DetectionDS(clean_data, noisy_data)
    criterion = nn.MSELoss(reduction='mean')

    for data_idx in list_of_indeces_to_check:
        data = dataset[data_idx]
        sample, target = torch.tensor(data[0]).unsqueeze(0).float(), torch.tensor(data[1]).unsqueeze(0).float()
        sample, target = sample.to(device) , target.to(device)

        prediction = model(sample)
        mel_prediction = sample - prediction
        mel_target = sample - target
        
        loss = criterion(mel_prediction, mel_target).item()
        
        sample = sample.squeeze(0).cpu().detach().numpy()
        mel_prediction = mel_prediction.squeeze(0).cpu().detach().numpy()
        mel_target = mel_target.squeeze(0).cpu().detach().numpy()
        print(f'mel-spectrogram idx={data_idx}, loss={loss:5.3}\n')
        
        plt.figure(figsize=(15,15))
        plt.subplot(311)
        plt.title(f'Noisy sample_{data_idx}')
        sns.heatmap(sample, cmap="YlGnBu")
        plt.subplot(312)
        plt.title(f'Predicted sample_{data_idx}')
        sns.heatmap(mel_prediction, cmap="YlGnBu")
        plt.subplot(313)
        plt.title(f'Clean sample_{data_idx}')
        sns.heatmap(mel_target, cmap="YlGnBu")
        pass

In [15]:
if len(list2visualize) > 0:
    EvaluateAndVisualize(loaded_denoising_model, test_clean_data, test_noisy_data, device, list2visualize)